# Déployez un modèle dans le cloud version AWS

## Importation des librairies 

In [1]:
import io
from io import StringIO
import os
import s3fs
#libraires classiques
import numpy as np
import pandas as pd

from PIL import Image,ImageOps,ImageFilter

#pyspark modules
import pyspark 
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, split
from pyspark.ml.linalg import Vectors, VectorUDT, DenseVector
from pyspark.sql.functions import udf, element_at
from pyspark.ml.image import ImageSchema

# tnesorflow modules
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1644825675241_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1644825675241_0001,pyspark,idle,Link,Link,None,✔


## Path

In [3]:
PATH = 's3://p8-agri'
PATH_df = PATH+'/Test'
PATH_resultat = PATH+'/Resultats'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Chargement des données 

In [4]:
image_df = spark.read.format("binaryFile").option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true").load(PATH_df)
print(image_df.printSchema())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)

None

In [5]:
image_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3://p8-agri/Test...|2022-02-11 07:45:18|  6894|[FF D8 FF E0 00 1...|
|s3://p8-agri/Test...|2022-02-11 07:45:18|  6880|[FF D8 FF E0 00 1...|
|s3://p8-agri/Test...|2022-02-11 07:45:18|  6857|[FF D8 FF E0 00 1...|
|s3://p8-agri/Test...|2022-02-11 07:44:58|  6854|[FF D8 FF E0 00 1...|
|s3://p8-agri/Test...|2022-02-11 07:44:58|  6849|[FF D8 FF E0 00 1...|
|s3://p8-agri/Test...|2022-02-11 07:44:58|  6848|[FF D8 FF E0 00 1...|
|s3://p8-agri/Test...|2022-02-11 07:44:58|  6842|[FF D8 FF E0 00 1...|
|s3://p8-agri/Test...|2022-02-11 07:44:58|  6834|[FF D8 FF E0 00 1...|
|s3://p8-agri/Test...|2022-02-11 07:44:58|  6831|[FF D8 FF E0 00 1...|
|s3://p8-agri/Test...|2022-02-11 07:45:18|  6814|[FF D8 FF E0 00 1...|
|s3://p8-agri/Test...|2022-02-11 07:44:58|  6796|[FF D8 FF E0 00 1...|
|s3://

In [6]:
def get_folder_name(path):
    #Returns fruit folder name
    ls_files = path.split('/')
    fruit = ls_files[-2] #folder name
    return fruit

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
from pyspark.sql.types import ArrayType, IntegerType, FloatType, StringType
udf_fruits = udf(get_folder_name, StringType())
image_df = image_df.withColumn('label', udf_fruits('PATH'))
image_df = image_df.withColumn('image', split(col('PATH'), '/').getItem(5))
image_df = image_df.select('PATH', 'content', 'label', 'image')
image_df.show(8)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-------------+-------------+
|                PATH|             content|        label|        image|
+--------------------+--------------------+-------------+-------------+
|s3://p8-agri/Test...|[FF D8 FF E0 00 1...|Cucumber Ripe|r_165_100.jpg|
|s3://p8-agri/Test...|[FF D8 FF E0 00 1...|Cucumber Ripe|r_163_100.jpg|
|s3://p8-agri/Test...|[FF D8 FF E0 00 1...|Cucumber Ripe|r_164_100.jpg|
|s3://p8-agri/Test...|[FF D8 FF E0 00 1...|  Cauliflower|r_183_100.jpg|
|s3://p8-agri/Test...|[FF D8 FF E0 00 1...|  Cauliflower|r_173_100.jpg|
|s3://p8-agri/Test...|[FF D8 FF E0 00 1...|  Cauliflower|r_181_100.jpg|
|s3://p8-agri/Test...|[FF D8 FF E0 00 1...|  Cauliflower|r_180_100.jpg|
|s3://p8-agri/Test...|[FF D8 FF E0 00 1...|  Cauliflower|r_185_100.jpg|
+--------------------+--------------------+-------------+-------------+
only showing top 8 rows

## Préparation du modèle

### Transfer-learning

In [8]:
model = ResNet50(include_top=False,
                weights=None,
                pooling='max',
                input_shape=(224,224,3),
                )
model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [9]:
bc_model_weights = spark.sparkContext.broadcast(model.get_weights()) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Fonction
Les fonctions qui suivent sont repris de ce site web 
https://docs.databricks.com/applications/machine-learning/preprocess-data/transfer-learning-tensorflow.html

+ Notre modèle de transfer-learning

In [10]:
def model_fn():
    
    model = ResNet50(include_top=False,
                weights=None,
                pooling='max',
                input_shape=(224,224,3)) 
    model.set_weights(bc_model_weights.value)
    return model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+ Preprocessing

In [11]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+ Prédiction du modèle donnat des features en sortie

In [12]:
def featurize_series(model, content_series):
    
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    
    input = np.stack(content_series.map(preprocess)) 
    preds = model.predict(input)
    output = [p.flatten() for p in preds]
    return pd.Series(output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+ Rassemblement en utilisant Pandas_udf sur notre fonction spark

In [13]:
@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).
  
    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                                is a pandas Series of image data.
    '''
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/pandas/functions.py:392: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

## Application des divers fonction pour l'extraction de features 

In [14]:
features_df = image_df.select(col("PATH"), col("label"), featurize_udf("content").alias("vecs"))
features_df.printSchema()
features_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- PATH: string (nullable = true)
 |-- label: string (nullable = true)
 |-- vecs: array (nullable = true)
 |    |-- element: float (containsNull = true)

+--------------------+-------------+--------------------+
|                PATH|        label|                vecs|
+--------------------+-------------+--------------------+
|s3://p8-agri/Test...|Cucumber Ripe|[8.608451, 10.534...|
|s3://p8-agri/Test...|Cucumber Ripe|[9.597337, 10.006...|
|s3://p8-agri/Test...|Cucumber Ripe|[9.8227625, 10.27...|
|s3://p8-agri/Test...|  Cauliflower|[8.268747, 9.0835...|
|s3://p8-agri/Test...|  Cauliflower|[6.541216, 9.9968...|
|s3://p8-agri/Test...|  Cauliflower|[7.4994335, 9.312...|
|s3://p8-agri/Test...|  Cauliflower|[7.20341, 9.35611...|
|s3://p8-agri/Test...|  Cauliflower|[8.271177, 8.3704...|
|s3://p8-agri/Test...|  Cauliflower|[8.246858, 8.7758...|
|s3://p8-agri/Test...|Cucumber Ripe|[9.176495, 9.9146...|
|s3://p8-agri/Test...|  Cauliflower|[8.497028, 8.4666...|
|s3://p8-agri/Test...|Cucum

## Enregistrement des résultats

In [15]:
features_df.write.mode("overwrite").parquet(PATH_resultat)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Chargement des résultats 

In [16]:
df = pd.read_parquet(PATH_resultat, engine='pyarrow')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
df.head(20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                             PATH  ...                                               vecs
0   s3://p8-agri/Test/Cucumber Ripe/r_165_100.jpg  ...  [8.608451, 10.534962, 17.155579, 11.381365, 0....
1   s3://p8-agri/Test/Cucumber Ripe/r_163_100.jpg  ...  [9.597337, 10.006106, 17.18625, 10.735639, 1.4...
2   s3://p8-agri/Test/Cucumber Ripe/r_164_100.jpg  ...  [9.8227625, 10.277847, 17.21304, 11.347143, 1....
3     s3://p8-agri/Test/Cauliflower/r_183_100.jpg  ...  [8.268747, 9.083594, 17.60324, 7.7839975, 1.22...
4     s3://p8-agri/Test/Cauliflower/r_173_100.jpg  ...  [6.541216, 9.996861, 18.364935, 8.431671, 1.15...
5     s3://p8-agri/Test/Cauliflower/r_181_100.jpg  ...  [7.4994335, 9.312448, 17.636358, 7.9417887, 1....
6     s3://p8-agri/Test/Cauliflower/r_180_100.jpg  ...  [7.20341, 9.356114, 17.420315, 7.595901, 1.167...
7     s3://p8-agri/Test/Cauliflower/r_185_100.jpg  ...  [8.271177, 8.370496, 17.475927, 8.206749, 1.21...
8     s3://p8-agri/Test/Cauliflower/r_184_100.

In [18]:
print("La taille du vecteur:", df.loc[1,'vecs'].shape)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

La taille du vecteur: (2048,)